In [13]:

import os
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from matplotlib import pyplot as plt
%matplotlib inline



In [14]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.12.0-cp310-cp310-win_amd64.whl (1.9 kB)
     -------------------------------------- 272.8/272.8 MB 2.1 MB/s eta 0:00:00
  Using cached libclang-16.0.0-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     -------------------------------------- 422.5/422.5 kB 3.7 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ---------------------------------------- 4.1/4.1 MB 3.6 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ---------------------------------------- 1.5/1.5 MB 2.5 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
     ---------------------------------------- 5.6/5.6 MB 3.3 MB/s eta 0:00:00
     ---------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [16]:
from PIL import Image as mg
import cv2
import numpy as np
import tensorflow as tf


In [17]:
WILD_PATH = 'wild'
IMAGE_SIZE = 128


In [18]:
# from google.colab import drive
# drive.mount('/content/drive')
# # Mounted at /content/drive
# Getting Images
images = os.listdir('/content/drive/MyDrive/imgs')
train_dir='/content/drive/MyDrive/imgs'



ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
trains = tf.keras.utils.image_dataset_from_directory(train_dir,validation_split=0.3,subset="training",seed=123,image_size=(IMAGE_SIZE, IMAGE_SIZE),)
vals = tf.keras.utils.image_dataset_from_directory(train_dir,validation_split=0.3,subset="validation",seed=123,image_size=(IMAGE_SIZE, IMAGE_SIZE),)
trains = trains.unbatch()
vals = vals.unbatch()
train = []
test = []
for x,y in trains:
    train.append(x.astype('float32')/255.)
for x,y in vals:
    test.append(x.astype('float32')/255.)
train = tf.convert_to_tensor(train)
test = tf.convert_to_tensor(test)
# Found 4772 files belonging to 1 classes.
# Using 3341 files for training.
# Found 4772 files belonging to 1 classes.
# Using 1431 files for validation.



In [ ]:
print(train.shape)
print(test.shape)
# (3341, 128, 128, 3)
# (1431, 128, 128, 3)
# Show Random Train Image:
# Convolutional Layers:


In [ ]:
#generating masks
left = 60
right = 76
up = 78
down = 62
x_train = train.numpy()
black_train = x_train.copy()
black_train[:,left:right,down:up,:] = 0
train_labels = x_train[:,left:right,down:up,:]
x_test = test.numpy()
black_test = x_test.copy()
black_test[:,left:right,down:up,:] = 0
test_labels = x_test[:,left:right,down:up,:]
plt.imshow(black_test[0])

In [ ]:

from tensorflow.keras.models import Model

encoder_in = tf.keras.Input(shape=(128,128,3),name='img_input')
layer = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoder_in)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoder_in)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Flatten()(layer)
encoder_out = tf.keras.layers.Dense(1024,activation='relu')(layer)     
decoder_in = tf.keras.layers.Dense(1024,activation='relu')(encoder_out)
layer = tf.keras.layers.Reshape((8,8,16))(decoder_in)
layer = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same')(layer)
layer = layer = tf.keras.layers.Conv2DTranspose(64,(3,3),activation='relu',padding='same')(layer)
layer = layer = tf.keras.layers.Conv2DTranspose(128,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same',strides=(2,2))(layer)
decoder_out = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same')(layer)

         




In [ ]:
autoencoder = tf.keras.Model(encoder_in,decoder_out,name='an_dec')
# autoencoder.summary()
      
optimizer = tf.keras.optimizers.Adam(lr = 0.001)
autoencoder.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
history = autoencoder.fit(black_train,train_labels ,
                epochs=50,
                shuffle=True,
                batch_size=10,
                validation_data=(black_test,test_labels))



In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()




In [ ]:
 
index = 145
ry = autoencoder.predict(black_test[index].reshape(-1,128,128,3))
pic = black_test[index].copy()
pic[left:right,down:up] = ry
rows = 1
colls = 3
fig = plt.figure(figsize=(10, 7))
fig.add_subplot(rows,colls,1)
plt.imshow(pic)
plt.axis('off')
plt.title("predicted")
fig.add_subplot(rows,colls,2)
plt.imshow(test_xs[index])
plt.axis('off')
plt.title("original")
fig.add_subplot(rows,colls,3)
plt.imshow(black_test[index])
plt.axis('off')
plt.title("input")
Text(0.5, 1.0, 'input')


In [5]:
#generating 32x32 holes
left = 52
right = 84
up = 86
down = 54
black_train_bigger = x_train.copy()
black_train_bigger[:,left:right,down:up,:] = 0
train_labels_bigger = x_train[:,left:right,down:up,:]
black_test_bigger = x_test.copy()
black_test_bigger[:,left:right,down:up,:] = 0
test_labels_bigger = x_test[:,left:right,down:up,:]
plt.imshow(black_train_bigger[0])
<matplotlib.image.AxesImage at 0x7f5243174650>



SyntaxError: invalid syntax (4125449092.py, line 13)

In [4]:
from tensorflow.keras.models import Model

encoder_in = tf.keras.Input(shape=(128,128,3),name='img_input')
layer = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoder_in)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoder_in)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Flatten()(layer)
encoder_out = tf.keras.layers.Dense(1024,activation='relu')(layer)
       
decoder_in = tf.keras.layers.Dense(1024,activation='relu')(encoder_out)
layer = tf.keras.layers.Reshape((8,8,16))(decoder_in)
layer = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same')(layer)
layer = layer = tf.keras.layers.Conv2DTranspose(64,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.Conv2DTranspose(128,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same',strides=(2,2))(layer)
decoder_out = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same',strides=(2,2))(layer)


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
autoencoder = tf.keras.Model(encoder_in,decoder_out,name='an_dec')
# autoencoder.summary()
optimizer = tf.keras.optimizers.Adam(lr = 0.001)
autoencoder.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
history = autoencoder.fit(black_train_bigger,train_labels_bigger ,
                epochs=50,
                shuffle=True,
                batch_size=10,
                validation_data=(black_test_bigger,test_labels_bigger))




NameError: name 'tf' is not defined

In [2]:
index = 23
ry = autoencoder.predict(black_test_bigger[index].reshape(-1,128,128,3))
pic = black_test[index].copy()
pic[left:right,down:up] = ry
rows = 1
colls = 3
fig = plt.figure(figsize=(10, 7))
fig.add_subplot(rows,colls,1)
plt.imshow(pic)
plt.axis('off')
plt.title("predicted")
fig.add_subplot(rows,colls,2)
plt.imshow(test[index])
plt.axis('off')
plt.title("original")
fig.add_subplot(rows,colls,3)
plt.imshow(black_test_bigger[index])
plt.axis('off')
plt.title("input")
Text(0.5, 1.0, 'input')

NameError: name 'autoencoder' is not defined

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
